## A Minimal "MultiWormTracker"

I don't like MATLAB, so I wrote this.

Given an input `.avi` file, this notebook will output the position for each worm at each frame in a `.csv` file. Does not extract any other features or extrapolate between frames where a worm is lost. Edit the parameters in ALL CAPS to fit to your data.

In [ ]:
from mwt import *

### 1 Locate files [EDIT]

The notebook requires a scratch directory with a lot of storage that will be filled with the `.avi` file converted into `.tif` for each individual frame. In my hands, 35 GB was used for a 45 min recording

In [ ]:
#EDIT, filenames with spaces don't quite work and so please replace with underscores first
INPUT_AVI = '/Volumes/UNTITLED/8.8.24_leaving_assay_N2_2.avi'
SCRATCH_PATH = '/Volumes/UNTITLED'
OUTPUT_PATH = '/Users/albertwang/Desktop'
FRAME_LIMIT = None

if not os.path.isfile(INPUT_AVI):
    raise FileNotFoundError()

### 2 Initialize scratch directory

In [ ]:
scratch_dir = make_scratch_dir(INPUT_AVI, SCRATCH_PATH)
scratch_dir

### 3 Convert `.avi` to `.tif` with `ffmpeg` (<5 min for a 45 min recording on my laptop)

In [ ]:
%%time

#comment if not needed
convert_to_tif(INPUT_AVI, scratch_dir, FRAME_LIMIT)

### 4 Generate a mask that's the mean out of an equally sampled 100 frames in the recording

In [ ]:
%%time

#comment if not needed
make_mask(scratch_dir)

#for plotting
mask = show_mask(scratch_dir)

### 5 Try out worm detection parameters first [EDIT] STOP AFTER THIS CELL TO MAKE SURE THE THRESHOLD PARAMETER IS RIGHT FOR THE DATA

This cell first detects blobs on a frame by subtracting each frame from the mean masking frame above and then thresholds that difference frame. A gaussian filter is also applied in between (which may actually be unnessary, but I feel makes the blob detection more robust). It then chooses which blobs are worms by just picking ones that are roughly the size of a worm and also uses an eccentricity cutoff remove any features that may be from the edge of the plate.

Definitely edit these parameters!

In [ ]:
#EDIT
SIGMA = 2 #can just be kept as two
THRESHOLD = 12 #THE MOST IMPORTANT PARAMETER AND NEEDS TO BE ADJUSTED FOR EACH VIDEO TO MAKE THE AVERAGE AREA OF A WORM ROUGHLY 100 px
#I'VE BEEN PICKING EVEN NUMBERS BETWEEN 8 and 14
AREA_RANGE = (25, 175) #can just be kept in this range
ECCENTRICITY_RANGE = (0, 0.99) #can also just be kept in this range (just meant to cut out any plate artifacts)

check_params(scratch_dir, mask, SIGMA, THRESHOLD, AREA_RANGE, ECCENTRICITY_RANGE, rows=10)

### 6 Detect worms for each frame (<6 min for a 45 min recording on my laptop)

In [ ]:
%%time

#comment if not needed
detect_worms(scratch_dir, mask, SIGMA, THRESHOLD, AREA_RANGE, ECCENTRICITY_RANGE)

#for plotting
plot_worms(scratch_dir)

### 7 Link `.csv` (<1 min for a 45 min recording on my laptop)

In [ ]:
%%time

#EDIT IF NEEDED (YOU PROBABLY DONT)
SEPERATION = 15

#comment if not needed
link_worms(scratch_dir, SEPERATION)

#for plotting
plot_untrimmed(scratch_dir)

### 8 Trim out short tracks

Removes tracks that are too short and move too little

In [ ]:
#EDIT IF NEEDED (YOU PROBABLY DONT)
MIN_FRAMES_SEEN = 100
MIN_AREA_TRAVELED = 20

#comment if not needed
trim_csv(scratch_dir, MIN_FRAMES_SEEN, MIN_AREA_TRAVELED)

#for plotting
plot_trimmed(scratch_dir)

### 9 One final visualization

In [ ]:
%%time

#for plotting
plot_tracks(scratch_dir, mask)

### 10 Copy all files to the output folder

In [ ]:
%%time

save_params(scratch_dir, SIGMA, THRESHOLD, AREA_RANGE, ECCENTRICITY_RANGE, SEPERATION, MIN_FRAMES_SEEN, MIN_AREA_TRAVELED)
copy_to_output(INPUT_AVI, OUTPUT_PATH, scratch_dir)